In [131]:
!pip uninstall lightgbm -y

Uninstalling lightgbm-2.3.1:
  Successfully uninstalled lightgbm-2.3.1


In [133]:
# !pip install scikit-learn==0.22
# !pip install setuptools wheel numpy scipy scikit-learn -U
!pip install lightgbm 

  Using cached https://files.pythonhosted.org/packages/0b/9d/ddcb2f43aca194987f1a99e27edf41cf9bc39ea750c3371c2a62698c509a/lightgbm-2.3.1-py2.py3-none-manylinux1_x86_64.whl
You should consider upgrading via the 'pip install --upgrade pip' command.


In [127]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sys
sys.path.append('../pp_1st')
import pp1st_pipeline
import pp1st_func
import pp2nd_func
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgbm
# from sklearn.svm import SVC
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 50)

In [2]:
data = pd.read_csv('../raw/PJT002_train.csv', )

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (8,172,173,174,175,176,177,178,179) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data['lnd_ar'].replace(0, np.nan, inplace=True)
data['ttl_ar'].replace(0, np.nan, inplace=True)
data['bldng_ar'].replace(0, np.nan, inplace=True)
data['wnd_drctn'].replace(0, np.nan, inplace=True)

In [4]:
data.dropna(axis=1, thresh=10000, inplace=True)

In [5]:
# data.dropna(axis=1, thresh=49000, inplace=True)

# base preprocess

In [5]:
#fr_yn
data.loc[data[data['fr_yn']=='Y'].index, 'fr_yn'] = 1
data.loc[data[data['fr_yn']=='N'].index, 'fr_yn'] = 0

In [6]:
data['emd_nm'] = data['emd_nm'].astype(str).apply(lambda x : x[4:] if x[:4]=='경상남도' else x )
data['emd_nm_big'] = data['emd_nm'].apply(lambda x : x.split()[0] if x!='nan' else x )
data['emd_nm_small'] = data['emd_nm'].apply(lambda x : x.split()[1] if x!='nan' else x)

In [7]:
data['dt_of_fr_yr'] = data['dt_of_fr'].astype('datetime64').apply(lambda x : x.year)
data['dt_of_fr_hr'] = data['dt_of_fr'].astype('datetime64').apply(lambda x : x.hour)
data['dt_of_fr_mth'] = data['dt_of_fr'].astype('datetime64').apply(lambda x : x.month)
data = pp1st_func.seasoning(data)
data.loc[data[data['season']=='봄'].index, 'season'] = 0
data.loc[data[data['season']=='여름'].index, 'season'] = 1
data.loc[data[data['season']=='가을'].index, 'season'] = 2
data.loc[data[data['season']=='겨울'].index, 'season'] = 3

In [8]:
data.loc[32635, 'dt_of_athrztn'] = 20020227 
data['dt_of_athrztn'].fillna(0, inplace=True)
data['dt_of_athrztn'] = data['dt_of_athrztn'].astype(float).astype(int)
data['dt_of_athrztn'] = data['dt_of_athrztn'].astype(str).apply(lambda x : x[:4] if len(x)==8 else x )
data['dt_of_athrztn'] = data['dt_of_athrztn'].astype(str).apply(lambda x : x[:-2] if len(x)==6 else x )
data['dt_of_athrztn'] = data['dt_of_athrztn'].astype(str).apply(lambda x : x[:4] if len(x)==7 else x )
data['dt_of_athrztn'] = data['dt_of_athrztn'].astype(str).apply(lambda x : '1'+ x[:3] if x[0]=='9' else x )
data['dt_of_athrztn'].replace('0', np.nan, inplace=True)

# data['dt_of_fr_yr'] = data['dt_of_fr'].astype('datetime64').apply(lambda x : x.year)
# data['dt_of_fr_yr'].replace(0, np.nan, inplace=True)
# data['화재시점나이'] = data['dt_of_fr_yr'] - data['dt_of_athrztn'].astype(int)
# data['화재시점나이'] = data['화재시점나이'].apply(lambda x : np.nan if len(str(x))==4 else x) # nan값은 그대로 nan값으로

In [9]:
data['dt_of_athrztn'].unique()

array(['1977', nan, '2000', '1936', '2015', '1994', '1947', '1935',
       '1992', '1930', '1980', '1952', '1997', '1943', '2010', '1971',
       '2002', '2003', '2013', '2014', '1988', '2004', '2016', '1995',
       '1989', '1925', '1951', '2008', '1999', '1934', '2009', '1993',
       '1915', '1920', '1967', '1957', '1945', '2006', '2011', '1979',
       '1984', '1955', '1991', '2005', '2007', '2018', '1998', '1976',
       '1926', '1946', '1990', '1959', '1940', '1996', '1958', '1953',
       '1965', '1970', '1972', '1960', '1982', '1954', '2012', '2017',
       '1931', '1912', '1975', '1942', '1900', '1969', '1962', '1950',
       '1939', '2001', '1961', '1983', '1974', '1985', '1973', '1987',
       '1964', '1941', '1956', '1963', '1966', '1938', '1978', '1913',
       '1923', '1949', '1986', '1933', '1901', '1944', '1890', '1968',
       '1905', '1981', '1916', '1898', '1924', '1910', '1914', '1932',
       '1850', '1948', '1927', '1937', '1917', '1904', '1878', '1921',
       '1

# 작은것부터 impute

In [8]:
data.reset_index(drop=True, inplace=True)

In [28]:
data.isnull().sum().sort_values()

dt_of_fr                    0
ahsm_dstnc                  0
sft_emrgnc_bll_dstnc        0
tbc_rtl_str_dstnc           0
cctv_in_100m                0
fr_wthr_fclt_in_100m        0
cctv_dstnc                  0
mlt_us_yn                   0
fr_wthr_fclt_dstnc          0
fr_sttn_dstnc               0
id                          0
jmk                         0
no_tbc_zn_dstnc             0
bldng_cnt_in_50m            0
fr_yn                       0
bldng_cnt                   0
emd_nm                      4
tmprtr                     13
hmdt                       22
wnd_spd                    33
fr_mn_cnt                  41
wnd_drctn                 221
hm_cnt                    701
rd_sd_nm                 1491
rgnl_ar_nm2              1491
                        ...  
gas_engry_us_201609     27581
ele_engry_us_201609     27581
gas_engry_us_201610     27581
ele_engry_us_201606     27581
gas_engry_us_201602     27581
ele_engry_us_201602     27581
ele_engry_us_201601     27581
ele_engry_

- hm_cnt(행정구역별로 연,월에 마다 갱신한다)

In [65]:
# data['emd_nm'] = data['emd_nm'].astype(str).apply(lambda x : x[4:] if x[:4]=='경상남도' else x )
# data['emd_nm_big'] = data['emd_nm'].apply(lambda x : x.split()[0] if x.split()[0]!='창원시' else x.split()[0]+x.split()[1])
# data['emd_nm_small'] = data['emd_nm'].apply(lambda x : x.split()[1] if (x.split()[0]!='창원시') & (x!='nan') else x)
# data['emd_nm_small'] = data['emd_nm_small'].apply(lambda x : x.split()[2] if x.split()[0]=='창원시' else x )

In [12]:
# data[data['emd_nm']=='nan'][['emd_nm','hm_cnt','dt_of_fr','dt_of_fr_yr','dt_of_fr_mth','fr_mn_cnt','fr_sttn_dstnc']]

,emd_nm,hm_cnt,dt_of_fr,dt_of_fr_yr,dt_of_fr_mth,fr_mn_cnt,fr_sttn_dstnc
17885,nan,NaN,2014-06-20 03:12,2014,6,194.0,1920
20869,nan,NaN,2014-03-16 16:17,2014,3,194.0,1443
52035,nan,NaN,2016-05-05 16:55,2016,5,194.0,872
58625,nan,NaN,2014-12-08 03:52,2014,12,139.0,3521


In [13]:
# data[data['fr_mn_cnt']==139][['emd_nm','fr_mn_cnt']]

,emd_nm,fr_mn_cnt
30,함안군 칠서면,139.0
84,함안군 가야읍,139.0
135,함안군 칠북면,139.0
213,함안군 가야읍,139.0
245,함안군 칠북면,139.0
249,함안군 칠서면,139.0
448,함안군 함안면,139.0
519,함안군 군북면,139.0
561,함안군 여항면,139.0
562,함안군 군북면,139.0


In [66]:
# data.loc[data[(data['emd_nm']=='nan') & (data['fr_mn_cnt']==194)].index, 'emd_nm_big'] = '진주시'
# data.loc[data[(data['emd_nm']=='nan') & (data['fr_mn_cnt']==139)].index, 'emd_nm_big'] = '함안군'

In [67]:
# emd_nm_big_le = LabelEncoder()
# emd_nm_small_le = LabelEncoder()
# data['emd_nm_big_le'] = emd_nm_big_le.fit_transform(data['emd_nm_big'])
# data['emd_nm_small_le'] = emd_nm_small_le.fit_transform(data['emd_nm_small'])

In [68]:
# for idx, row in data[data['hm_cnt'].isnull()].iterrows():
#     mean_value = data[(data['emd_nm_big_le']==row['emd_nm_big_le']) & (data['emd_nm_small_le']==row['emd_nm_small_le'])]['hm_cnt'].mean()
#     data.loc[idx, 'hm_cnt'] = mean_value

In [9]:
hm_lnd_imputer = IterativeImputer(max_iter=30)
hm_lnd_imp_res = hm_lnd_imputer.fit_transform(data[['dt_of_fr_yr','dt_of_fr_mth','fr_mn_cnt','fr_sttn_dstnc', 'hm_cnt']])
imp_hm_lnd = pd.DataFrame(hm_lnd_imp_res, columns =['dt_of_fr_yr','dt_of_fr_mth','fr_mn_cnt','fr_sttn_dstnc', 'hm_cnt'])

In [140]:
# imp[imp['hm_cnt'].isnull()][['emd_nm_big_le','emd_nm_small_le','dt_of_fr_yr','dt_of_fr_mth','fr_mn_cnt','fr_sttn_dstnc', 'hm_cnt']].index

,emd_nm_big_le,emd_nm_small_le,dt_of_fr_yr,dt_of_fr_mth,fr_mn_cnt,fr_sttn_dstnc,hm_cnt
1110,5,173,2017,11,164.0,4600,NaN
1327,11,244,2015,1,176.0,1792,NaN
1419,11,100,2014,4,176.0,1886,NaN
1522,11,229,2014,8,176.0,1846,NaN
1528,11,229,2016,2,176.0,1802,NaN
1529,11,229,2014,2,176.0,1781,NaN
1531,11,244,2015,10,176.0,1525,NaN
1557,11,171,2014,7,176.0,1985,NaN
2180,11,171,2016,11,176.0,1809,NaN
4023,12,292,2015,10,176.0,1307,NaN


In [141]:
# imp_hm_lnd.loc[imp[imp['hm_cnt'].isnull()][['emd_nm_big_le','emd_nm_small_le','dt_of_fr_yr','dt_of_fr_mth','fr_mn_cnt','fr_sttn_dstnc', 'hm_cnt']].index][['hm_cnt']]

,hm_cnt
1110,12352.172142
1327,15374.195207
1419,17947.758991
1522,15688.460477
1528,15711.710822
1529,15631.385559
1531,15818.823582
1557,16601.189290
2180,16956.281952
4023,15284.544345


In [10]:
data_ = data.drop(columns = ['dt_of_fr_yr','dt_of_fr_mth','fr_mn_cnt','fr_sttn_dstnc', 'hm_cnt'])
data_ = pd.concat([data_, imp_hm_lnd], axis=1)

- tmptr, hmdt, wnd_drctn, wnd_spd

In [12]:
tmprtr_imputer = IterativeImputer(max_iter=50)

In [15]:
# imp_data_tmp = data[data['tmprtr'].isnull()]
imp_data_tmp = data_[['fr_yn','season','계절평균온도차','hmdt','wnd_drctn','wnd_spd','tmprtr']]
# imp_data_tmp.dropna(inplace=True)

In [16]:
tmprtr_imp_res = tmprtr_imputer.fit_transform(imp_data_tmp)
imp_season = pd.DataFrame(tmprtr_imp_res, columns=imp_data_tmp.columns)

In [17]:
data__ = data_.drop(columns = ['fr_yn','season','계절평균온도차','hmdt','wnd_drctn','wnd_spd','tmprtr'])
data_ = pd.concat([data__, imp_season], axis=1)

In [18]:
data_.isnull().sum().sort_values()[25:]

wnd_drctn                  0
jmk                        0
wnd_spd                    0
tmprtr                     0
bldng_cnt                  0
rd_sd_nm                1491
rgnl_ar_nm2             1491
rgnl_ar_nm              1491
lnd_us_sttn_nm          1776
ttl_ar                  8426
bldng_ar                9660
ttl_grnd_flr           10210
ttl_dwn_flr            11005
bldng_ar_prc           21895
lnd_ar                 25569
ele_engry_us_201803    27581
gas_engry_us_201804    27581
gas_engry_us_201410    27581
gas_engry_us_201805    27581
gas_engry_us_201803    27581
gas_engry_us_201806    27581
ele_engry_us_201806    27581
gas_engry_us_201807    27581
ele_engry_us_201804    27581
ele_engry_us_201805    27581
                       ...  
ele_engry_us_201612    27581
gas_engry_us_201409    27581
ele_engry_us_201611    27581
gas_engry_us_201602    27581
ele_engry_us_201610    27581
gas_engry_us_201611    27581
ele_engry_us_201609    27581
gas_engry_us_201603    27581
ele_engry_us_2

In [19]:
data_ = data_.dropna(subset=['ttl_ar','bldng_ar'])
data_.drop(columns = 'rgnl_ar_nm2', inplace=True)

- rd_sd_nm (baysian_ridge)

In [20]:
data_.reset_index(drop=True, inplace=True)

In [21]:
jmk_le = LabelEncoder()
data_['jmk_le'] = jmk_le.fit_transform(data_['jmk'])
data_['mlt_us_yn'] = data_['mlt_us_yn'].apply(lambda x : 1 if x=='Y' else 0)
data_.loc[data_[data_['rd_sd_nm']=='지정되지않음'].index, 'rd_sd_nm'] = np.nan

In [27]:
# notna_col = data_.isnull().sum().sort_values()[3:34].keys()
# data_.isnull().sum().sort_values()[:36]

dt_of_fr                  0
id                        0
emd_nm                    0
fr_wthr_fclt_dstnc        0
mlt_us_yn                 0
cctv_dstnc                0
fr_wthr_fclt_in_100m      0
cctv_in_100m              0
tbc_rtl_str_dstnc         0
sft_emrgnc_bll_dstnc      0
ahsm_dstnc                0
no_tbc_zn_dstnc           0
bldng_cnt_in_50m          0
emd_nm_big                0
jmk                       0
emd_nm_small              0
emd_nm_small_le           0
dt_of_fr_yr               0
dt_of_fr_mth              0
fr_mn_cnt                 0
fr_sttn_dstnc             0
hm_cnt                    0
fr_yn                     0
season                    0
계절평균온도차                   0
hmdt                      0
wnd_drctn                 0
wnd_spd                   0
tmprtr                    0
dt_of_fr_hr               0
emd_nm_big_le             0
jmk_le                    0
bldng_cnt                 0
bldng_ar                  0
ttl_ar                    0
rgnl_ar_nm          

In [64]:
notna_col = ['fr_wthr_fclt_dstnc', 'mlt_us_yn', 'cctv_dstnc', 'fr_wthr_fclt_in_100m','cctv_in_100m', 'tbc_rtl_str_dstnc', 
             'sft_emrgnc_bll_dstnc','ahsm_dstnc', 'no_tbc_zn_dstnc', 'bldng_cnt_in_50m', 
#              'emd_nm_small_le', 'emd_nm_big_le',
             'fr_mn_cnt', 'fr_sttn_dstnc', 'hm_cnt', 'fr_yn', 'season', '계절평균온도차','hmdt', 'wnd_drctn', 'wnd_spd', 'tmprtr', 'dt_of_fr_hr', 
             'jmk_le', 'bldng_cnt', 'bldng_ar']
notna_col.append('rd_sd_nm_le')

In [62]:
notna_data = data_[data_['rd_sd_nm'].notna()]

In [25]:
rd_sd_nm_le = LabelEncoder()
notna_data_le = rd_sd_nm_le.fit_transform(notna_data['rd_sd_nm'])

In [26]:
data_['rd_sd_nm_le']=np.nan
data_.loc[notna_data.index, 'rd_sd_nm_le'] = notna_data_le

In [27]:
rd_sd_nm_imputer = IterativeImputer(max_iter=50)
rd_sd_nm_res = rd_sd_nm_imputer.fit_transform(data_[notna_col])
rd_sd_nm_df = pd.DataFrame(rd_sd_nm_res,  columns = notna_col)

In [29]:
data__ = pd.concat([data_.drop(columns='rd_sd_nm_le'), rd_sd_nm_df[['rd_sd_nm_le']]], axis=1)

In [40]:
data__.isnull().sum().sort_values()[:39]

dt_of_fr                   0
id                         0
emd_nm                     0
fr_wthr_fclt_dstnc         0
mlt_us_yn                  0
cctv_dstnc                 0
fr_wthr_fclt_in_100m       0
cctv_in_100m               0
tbc_rtl_str_dstnc          0
sft_emrgnc_bll_dstnc       0
ahsm_dstnc                 0
no_tbc_zn_dstnc            0
bldng_cnt_in_50m           0
emd_nm_big                 0
emd_nm_small               0
dt_of_fr_hr                0
emd_nm_small_le            0
dt_of_fr_yr                0
dt_of_fr_mth               0
fr_mn_cnt                  0
fr_sttn_dstnc              0
hm_cnt                     0
fr_yn                      0
season                     0
계절평균온도차                    0
hmdt                       0
wnd_drctn                  0
wnd_spd                    0
tmprtr                     0
emd_nm_big_le              0
jmk                        0
jmk_le                     0
rd_sd_nm_le                0
bldng_cnt                  0
bldng_ar      

In [30]:
data__.to_csv('./data/진행중_rd_sd_nm끝.csv')

- rgnl_ar_nm

In [31]:
data = pd.read_csv('./data/진행중_rd_sd_nm끝.csv', index_col=0)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [32]:
data.reset_index(drop=True, inplace=True)

In [33]:
notna_data = data[data['rgnl_ar_nm'].notna()]
rgnl_ar_nm_le = LabelEncoder()
notna_data_le = rgnl_ar_nm_le.fit_transform(notna_data['rgnl_ar_nm'])

In [34]:
data['rgnl_ar_nm_le']= np.nan
data.loc[notna_data.index, 'rgnl_ar_nm_le'] = notna_data_le

In [35]:
notna_col.append('rd_sd_nm_le')
# notna_col.append('rgnl_ar_nm_le')

In [36]:
rf_rgnl_ar = RandomForestClassifier()

In [92]:
# rgnl_ar_nm_imputer = IterativeImputer(estimator= rf, max_iter=50)
# rgnl_ar_nm_res = rgnl_ar_nm_imputer.fit_transform(data_[notna_col])
# rgnl_ar_nm_df = pd.DataFrame(rgnl_ar_nm_res,  columns = notna_col)

KeyboardInterrupt: 

In [37]:
rf_rgnl_ar.fit(notna_data[notna_col], data.loc[notna_data.index,'rgnl_ar_nm_le'])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [38]:
rf_yhat = rf_rgnl_ar.predict(data[data['rgnl_ar_nm_le'].isnull()][notna_col])

In [39]:
data.loc[data[data['rgnl_ar_nm_le'].isnull()].index, 'rgnl_ar_nm_le'] = rf_yhat

- lnd_us_sttn_nm

In [42]:
data.reset_index(drop=True, inplace=True)

In [43]:
notna_data = data[data['lnd_us_sttn_nm'].notna()]
lnd_us_sttn_nm_le = LabelEncoder()
notna_data_le = lnd_us_sttn_nm_le.fit_transform(notna_data['lnd_us_sttn_nm'])
data['lnd_us_sttn_nm_le']= np.nan
data.loc[notna_data.index, 'lnd_us_sttn_nm_le'] = notna_data_le

In [44]:
notna_col.append('rgnl_ar_nm_le')

In [45]:
rf_lnd_us_sttn = RandomForestClassifier()
rf_lnd_us_sttn.fit(notna_data[notna_col], data.loc[notna_data.index,'lnd_us_sttn_nm_le'])
rf_yhat = rf_lnd_us_sttn.predict(data[data['lnd_us_sttn_nm_le'].isnull()][notna_col])

In [46]:
data.loc[data[data['lnd_us_sttn_nm_le'].isnull()].index, 'lnd_us_sttn_nm_le'] = rf_yhat

In [36]:
data.isnull().sum().sort_values()[:45]

dt_of_fr                    0
id                          0
fr_wthr_fclt_dstnc          0
mlt_us_yn                   0
cctv_dstnc                  0
fr_wthr_fclt_in_100m        0
cctv_in_100m                0
tbc_rtl_str_dstnc           0
sft_emrgnc_bll_dstnc        0
ahsm_dstnc                  0
no_tbc_zn_dstnc             0
bldng_cnt_in_50m            0
emd_nm_big                  0
dt_of_fr_hr                 0
emd_nm_small_le             0
dt_of_fr_yr                 0
dt_of_fr_mth                0
fr_mn_cnt                   0
fr_sttn_dstnc               0
hm_cnt                      0
fr_yn                       0
season                      0
계절평균온도차                     0
hmdt                        0
wnd_drctn                   0
wnd_spd                     0
tmprtr                      0
emd_nm_big_le               0
jmk_le                      0
rd_sd_nm_le                 0
jmk                         0
rgnl_ar_nm_le               0
lnd_us_sttn_nm_le           0
bldng_cnt 

In [47]:
data.to_csv('./data/진행중_lnd_us_sttn_nm끝.csv')

In [154]:
data = pd.read_csv('./data/진행중_lnd_us_sttn_nm끝.csv', index_col=0)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


- ttl_grnd_flr, ttl_dwn_flr

In [158]:
data.loc[data[(data['ttl_grnd_flr'].isnull()) & (data['bldng_ar'] == data['ttl_ar']) & (data['bldng_ar']!=0)].index, 'ttl_grnd_flr'] = 0
data.loc[data[(data['ttl_dwn_flr'].isnull()) & (data['bldng_ar'] == data['ttl_ar']) & (data['bldng_ar']!=0)].index, 'ttl_dwn_flr'] = 0

In [173]:
rf_ttl_flr = RandomForestClassifier()
# rf_lnd_us_sttn.fit(notna_data[notna_col], data.loc[notna_data.index,'lnd_us_sttn_nm_le'])
# rf_yhat = rf_lnd_us_sttn.predict(data[data['lnd_us_sttn_nm_le'].isnull()][notna_col])

In [174]:
ttl_flr_imputer = IterativeImputer(estimator= rf_ttl_flr , max_iter=50)
ttl_flr_res = ttl_flr_imputer.fit_transform(data[notna_col])
ttl_flr_df = pd.DataFrame(ttl_flr_res, columns = notna_col)

In [177]:
data_ = pd.concat([data.drop(columns=['ttl_grnd_flr','ttl_dwn_flr']) , ttl_flr_df[['ttl_grnd_flr','ttl_dwn_flr']]], axis=1)

In [182]:
data_.to_csv('./data/진행중_ttl_flr끝.csv')

In [181]:
data_.isnull().sum().sort_values()[35:]

emd_nm_big                 3
emd_nm_small               3
emd_nm                     3
rgnl_ar_nm               878
rd_sd_nm                 879
lnd_us_sttn_nm          1147
bldng_ar_prc           15362
lnd_ar                 16569
ele_engry_us_201806    19144
gas_engry_us_201807    19144
ele_engry_us_201807    19144
ele_engry_us_201412    19144
gas_engry_us_201806    19144
ele_engry_us_201808    19144
gas_engry_us_201809    19144
ele_engry_us_201809    19144
gas_engry_us_201808    19144
ele_engry_us_201805    19144
gas_engry_us_201804    19144
ele_engry_us_201804    19144
gas_engry_us_201810    19144
ele_engry_us_201803    19144
gas_engry_us_201803    19144
ele_engry_us_201802    19144
gas_engry_us_201802    19144
                       ...  
gas_engry_us_201703    19144
ele_engry_us_201702    19144
ele_engry_us_201411    19144
ele_engry_us_201604    19144
gas_engry_us_201701    19144
ele_engry_us_201701    19144
gas_engry_us_201612    19144
ele_engry_us_201605    19144
gas_engry_us_2

- lnd_ar

In [191]:
data_[data_['lnd_ar'].isnull()][['bldng_ar','bldng_cnt','ttl_ar','lnd_ar','jmk','rgnl_ar_nm','bldng_archtctr','bldng_us_clssfctn','bldng_us','lnd_us_sttn_nm','mlt_us_yn']]

,bldng_ar,bldng_cnt,ttl_ar,lnd_ar,jmk,rgnl_ar_nm,bldng_archtctr,bldng_us_clssfctn,bldng_us,lnd_us_sttn_nm,mlt_us_yn
0,69.420,3,69.4200,NaN,대,자연녹지지역,블록구조,주거용,단독주택,단독,0
1,46.290,3,46.2900,NaN,대,계획관리지역,NaN,NaN,NaN,단독,0
3,48.920,2,48.9200,NaN,대,제1종일반주거지역,일반목구조,주거용,단독주택,단독,0
6,92.000,6,92.0400,NaN,답,계획관리지역,벽돌구조,주거용,단독주택,단독,0
7,102.360,8,102.3600,NaN,대,자연녹지지역,NaN,NaN,NaN,단독,0
8,66.130,4,66.1300,NaN,대,계획관리지역,일반목구조,주거용,단독주택,단독,0
10,44.970,3,44.9700,NaN,대,제2종일반주거지역,NaN,NaN,NaN,단독,0
11,1046.400,3,12067.3800,NaN,대,제2종일반주거지역,철근콘크리트구조,주거용,공동주택,아파트,0
13,66.120,4,66.1200,NaN,대,제2종일반주거지역,일반목구조,주거용,단독주택,단독,0
16,49.580,3,49.5800,NaN,대,제2종일반주거지역,일반목구조,주거용,단독주택,단독,0


In [193]:
data_['bldng_ar_cnt'] = data_['bldng_ar'] * data_['bldng_cnt']

In [291]:
data_.rename(columns={'계절평균온도차' : 'season_temperature_mean'}, inplace=True)

In [286]:
# notna_col.append('lnd_us_sttn_nm_le')
notna_col.append('bldng_ar_cnt')

In [287]:
notna_data = data_[data_['lnd_ar'].notna()][notna_col]

In [268]:
# lnd_us_sttn_le = LabelEncoder()
# notna_data['lnd_us_sttn_nm_le'] = lnd_us_sttn_le.fit_transform(notna_data['lnd_us_sttn_nm'])

TypeError: argument must be a string or number

In [289]:
lnd_ar_lgbm = lgbm.LGBMRegressor()
lnd_ar_lgbm.fit(X = notna_data, y = data_.loc[notna_data.index,'lnd_ar'])
lnd_ar_yhat = lnd_ar_lgbm.predict(data_[data_['lnd_ar'].isnull()][notna_col])
data_.loc[data_[data_['lnd_ar'].isnull()].index, 'lnd_ar'] = lnd_ar_yhat

ValueError: Input data must be 2 dimensional and non empty.

- bldng_ar_prc

In [295]:
# data_.rename(columns={'season_tempererature_mean' : 'season_temperature_mean'}, inplace=True)

In [297]:
# notna_col.remove('계절평균온도차')
# notna_col.remove('season_tempererature_mean')
notna_col.append('season_temperature_mean')

In [298]:
notna_col.append('lnd_ar')
notna_data = data_[data_['bldng_ar_prc'].notna()][notna_col]

In [299]:
bldng_ar_prc_lgbm = lgbm.LGBMRegressor()
bldng_ar_prc_lgbm.fit(X = notna_data, y = data_.loc[notna_data.index,'bldng_ar_prc'])
bldng_ar_yhat = bldng_ar_prc_lgbm.predict(data_[data_['bldng_ar_prc'].isnull()][notna_col])
data_.loc[data_[data_['bldng_ar_prc'].isnull()].index, 'bldng_ar_prc'] = bldng_ar_yhat

In [302]:
notna_col.append('bldng_ar_prc')

In [301]:
data_.to_csv('./data/진행중_bldng_ar_prc끝.csv')

- ele_engry_us

In [300]:
data_.isnull().sum().sort_values()[35:]

bldng_ar                   0
ttl_ar                     0
lnd_ar                     0
emd_nm_big                 3
emd_nm                     3
emd_nm_small               3
rgnl_ar_nm               878
rd_sd_nm                 879
lnd_us_sttn_nm          1147
ele_engry_us_201712    19144
ele_engry_us_201810    19144
gas_engry_us_201810    19144
gas_engry_us_201711    19144
ele_engry_us_201809    19144
gas_engry_us_201809    19144
ele_engry_us_201808    19144
gas_engry_us_201808    19144
ele_engry_us_201807    19144
gas_engry_us_201807    19144
ele_engry_us_201806    19144
gas_engry_us_201806    19144
ele_engry_us_201805    19144
gas_engry_us_201805    19144
ele_engry_us_201804    19144
gas_engry_us_201811    19144
                       ...  
gas_engry_us_201704    19144
ele_engry_us_201703    19144
gas_engry_us_201703    19144
ele_engry_us_201605    19144
ele_engry_us_201412    19144
ele_engry_us_201702    19144
gas_engry_us_201701    19144
ele_engry_us_201606    19144
gas_engry_us_2

# 모델링시도

In [113]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

In [370]:
# chosen_col = data_.isnull().sum().sort_values()[2: 41].keys()
chosen_col = list(chosen_col)

In [377]:
chosen_col.remove('fr_yn')

In [366]:
data_[chosen_col]

,bldng_ar_prc,fr_wthr_fclt_dstnc,mlt_us_yn,cctv_dstnc,fr_wthr_fclt_in_100m,cctv_in_100m,tbc_rtl_str_dstnc,sft_emrgnc_bll_dstnc,ahsm_dstnc,no_tbc_zn_dstnc,bldng_cnt_in_50m,dt_of_fr_hr,dt_of_fr_yr,jmk,dt_of_fr_mth,fr_sttn_dstnc,hm_cnt,fr_yn,season,season_temperature_mean,hmdt,wnd_drctn,wnd_spd,tmprtr,jmk_le,rd_sd_nm_le,rgnl_ar_nm_le,lnd_us_sttn_nm_le,ttl_grnd_flr,fr_mn_cnt,ttl_dwn_flr,bldng_ar_cnt,bldng_cnt,bldng_ar,ttl_ar,lnd_ar,emd_nm_big,emd_nm,emd_nm_small
0,4.718420e+05,133,0,112,0,0,1891,29231,11322,88,0,5,2017.0,대,10.0,4547.0,17360.0,1.0,2.0,-4.716313,96.0,200.000000,0.9,10.7,4,7.0,13.0,11.0,1.000000,137.0,0.000000,208.260,3,69.420,69.4200,4.207569e+02,창녕군,창녕군 창녕읍,창녕읍
1,1.225810e+05,489,0,818,0,0,4533,20480,3369,7727,0,8,2018.0,대,9.0,6388.0,1791.0,0.0,2.0,4.083687,74.0,20.000000,0.5,19.5,4,6.0,1.0,11.0,1.000000,85.0,0.000000,138.870,3,46.290,46.2900,4.207569e+02,의령군,의령군 가례면,가례면
2,6.181050e+05,143,0,165,0,0,277,31197,12451,72,14,14,2016.0,답,10.0,3340.0,17285.0,1.0,2.0,0.083687,21.0,90.000000,2.0,15.5,3,11.0,17.0,23.0,5.000000,137.0,0.000000,583.800,1,583.800,2516.7600,1.446000e+03,창녕군,창녕군 창녕읍,창녕읍
3,7.195420e+05,1585,0,131,0,0,438,4270,407,508,11,5,2016.0,대,6.0,179.0,7327.0,0.0,1.0,-4.137440,91.0,180.146290,0.4,20.6,4,7.0,15.0,11.0,1.000000,176.0,0.000000,97.840,2,48.920,48.9200,4.207569e+02,창원시의창구,창원시의창구 대산면,대산면
4,8.990870e+05,294,0,210,0,0,1425,29709,11690,206,0,15,2018.0,대,4.0,4278.0,17278.0,1.0,0.0,13.906737,11.0,180.000000,4.5,27.8,4,6.0,13.0,11.0,2.000000,137.0,0.000000,96.660,1,96.660,149.0400,4.860000e+02,창녕군,창녕군 창녕읍,창녕읍
5,7.107637e+05,778,0,741,0,0,3128,29118,7676,1609,0,1,2015.0,대,9.0,4387.0,17132.0,0.0,2.0,5.483687,93.0,360.000000,0.5,20.9,4,9.0,1.0,33.0,1.000000,137.0,0.000000,84.000,1,84.000,84.0000,4.950000e+02,창녕군,창녕군 창녕읍,창녕읍
6,2.335940e+05,310,0,519,0,0,1951,33829,13237,695,0,23,2018.0,답,3.0,2008.0,17302.0,0.0,0.0,-8.093263,78.0,200.000000,0.5,5.8,3,9.0,1.0,11.0,1.000000,137.0,0.000000,552.000,6,92.000,92.0400,4.207569e+02,창녕군,창녕군 창녕읍,창녕읍
7,5.169770e+05,1027,0,521,0,0,1201,30379,9565,523,5,8,2018.0,대,3.0,2752.0,17302.0,0.0,0.0,-11.793263,95.0,180.000000,0.8,2.1,4,9.0,13.0,11.0,1.000000,137.0,0.000000,818.880,8,102.360,102.3600,6.421882e+02,창녕군,창녕군 창녕읍,창녕읍
8,2.815741e+05,1326,0,460,0,0,3528,26904,10588,133,0,5,2014.0,대,12.0,6587.0,17199.0,0.0,3.0,-5.471167,89.0,70.000000,0.8,-2.8,4,6.0,1.0,11.0,1.000000,137.0,0.000000,264.520,4,66.130,66.1300,4.207569e+02,창녕군,창녕군 창녕읍,창녕읍
9,2.631170e+05,66,0,46,1,2,135,31585,12101,18,7,8,2014.0,대,12.0,2583.0,17199.0,1.0,3.0,-2.571167,66.0,229.619036,0.6,0.1,4,7.0,17.0,11.0,1.000000,137.0,0.000000,197.920,2,98.960,98.9600,3.100000e+02,창녕군,창녕군 창녕읍,창녕읍


In [378]:
x = data_[chosen_col]
y = data_['fr_yn']

In [379]:
x['mlt_us_yn'] = x['mlt_us_yn'].astype('category')
x['season'] = x['season'].astype('category')
x['dt_of_fr_hr'] = x['dt_of_fr_hr'].astype('category')
x['dt_of_fr_mth'] = x['dt_of_fr_mth'].astype('category')
x['jmk_le'] = x['jmk_le'].astype('category')
x['rd_sd_nm_le'] = x['rd_sd_nm_le'].astype('category')
x['rgnl_ar_nm_le'] = x['rgnl_ar_nm_le'].astype('category')
x['lnd_us_sttn_nm_le'] = x['lnd_us_sttn_nm_le'].astype('category')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [346]:
# notna_col.append('lnd_ar')

In [108]:
xgb = XGBClassifier(max_depth=20)

In [224]:
stf = StratifiedKFold(n_splits=5, shuffle=True)

In [110]:
cv_score = cross_val_score(xgb, X = x, y = y, cv=stf)
cv_score

array([0.85934976, 0.85743134, 0.86015751, 0.86004241, 0.85802282])

In [347]:
xgb.fit(x,y)

ValueError: DataFrame.dtypes for data must be int, float or bool.
                Did not expect the data types in fields mlt_us_yn, season, dt_of_fr_hr, jmk_le, rd_sd_nm_le, rgnl_ar_nm_le, lnd_us_sttn_nm_le, dt_of_fr_mth

In [142]:
# x.rename(columns={'계절평균온도차' : 'season_mean' }, inplace=True)

In [333]:
lgbm_ = lgbm.LGBMClassifier()
cv_score = cross_val_score(lgbm_, X = x, y = y, #cv=stf,
                            scoring='f1')
cv_score

array([0.29548989, 0.25294748, 0.31889964, 0.34482759, 0.39965473])

In [304]:
sklearn.metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted'])

In [380]:
lgbm_.fit(x,y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [381]:
yhat = lgbm_.predict(x)

In [382]:
f1_score(y, yhat)

0.42924159499609066

# validation

In [48]:
data_val = pd.read_csv('../raw/PJT002_validation.csv' )

In [49]:
data_val = pp2nd_func.replace(data_val)
data_val = pp2nd_func.label_encoding(data_val)
data_val = pp2nd_func.emd_nm_encoding(data_val)

In [50]:
data_val.reset_index(drop=True, inplace=True)

In [51]:
hm_lnd_imp_res = hm_lnd_imputer.transform(data_val[['dt_of_fr_yr','dt_of_fr_mth','fr_mn_cnt','fr_sttn_dstnc', 'hm_cnt']])
imp_hm_lnd = pd.DataFrame(hm_lnd_imp_res, columns =['dt_of_fr_yr','dt_of_fr_mth','fr_mn_cnt','fr_sttn_dstnc', 'hm_cnt'])

In [52]:
data_val_ = data_val.drop(columns = ['dt_of_fr_yr','dt_of_fr_mth','fr_mn_cnt','fr_sttn_dstnc', 'hm_cnt'])
data_val_ = pd.concat([data_val_, imp_hm_lnd], axis=1)

In [57]:
# 계절변수들
imp_data_tmp = data_val_[['fr_yn','season','계절평균온도차','hmdt','wnd_drctn','wnd_spd','tmprtr']]
tmprtr_imp_res = tmprtr_imputer.transform(imp_data_tmp)
imp_season = pd.DataFrame(tmprtr_imp_res, columns=imp_data_tmp.columns)

In [58]:
data_val__ = data_val_.drop(columns = ['fr_yn','season','계절평균온도차','hmdt','wnd_drctn','wnd_spd','tmprtr'])
data_val_ = pd.concat([data_val__, imp_season], axis=1)

In [60]:
data_val_['jmk_le'] = jmk_le.transform(data_val_['jmk'])
data_val_['mlt_us_yn'] = data_val_['mlt_us_yn'].apply(lambda x : 1 if x=='Y' else 0)
data_val_.loc[data_val_[data_val_['rd_sd_nm']=='지정되지않음'].index, 'rd_sd_nm'] = np.nan

In [67]:
# notna_data = data_val_[data_val_['rd_sd_nm'].notna()]
# notna_data_le = rd_sd_nm_le.transform(notna_data['rd_sd_nm'])
data_val_['rd_sd_nm_le']=np.nan
data_val_.loc[notna_data.index, 'rd_sd_nm_le'] = notna_data_le

In [68]:
rd_sd_nm_res = rd_sd_nm_imputer.transform(data_val_[notna_col])
rd_sd_nm_df = pd.DataFrame(rd_sd_nm_res,  columns = notna_col)
data_val_ = pd.concat([data_val_.drop(columns='rd_sd_nm_le'), rd_sd_nm_df[['rd_sd_nm_le']]], axis=1)

In [69]:
data_val_.reset_index(drop=True, inplace=True)

In [70]:
notna_data = data_val_[data_val_['rgnl_ar_nm'].notna()]
notna_data_le = rgnl_ar_nm_le.transform(notna_data['rgnl_ar_nm'])
data_val_['rgnl_ar_nm_le']= np.nan
data_val_.loc[notna_data.index, 'rgnl_ar_nm_le'] = notna_data_le

In [71]:
notna_col.append('rd_sd_nm_le')
rf_yhat = rf_rgnl_ar.predict(data_val_[data_val_['rgnl_ar_nm_le'].isnull()][notna_col])

In [72]:
data_val_.loc[data_val_[data_val_['rgnl_ar_nm_le'].isnull()].index, 'rgnl_ar_nm_le'] = rf_yhat

In [79]:
data_val_.drop(index = data_val_[data_val_['lnd_us_sttn_nm']=='경마장'].index, inplace=True)

In [80]:
notna_data = data_val_[data_val_['lnd_us_sttn_nm'].notna()]
notna_data_le = lnd_us_sttn_nm_le.transform(notna_data['lnd_us_sttn_nm'])
data_val_['lnd_us_sttn_nm_le']= np.nan
data_val_.loc[notna_data.index, 'lnd_us_sttn_nm_le'] = notna_data_le

In [81]:
notna_col.append('rgnl_ar_nm_le')

In [82]:
rf_yhat = rf_lnd_us_sttn.predict(data_val_[data_val_['lnd_us_sttn_nm_le'].isnull()][notna_col])
data_val_.loc[data_val_[data_val_['lnd_us_sttn_nm_le'].isnull()].index, 'lnd_us_sttn_nm_le'] = rf_yhat

In [83]:
data_val_.to_csv('./data/data_val_lnd_us_sttn끝.csv')

In [112]:
x_val = data_val_[notna_col].drop(columns='fr_yn')
y_val = data_val_['fr_yn']
yhat_val = xgb.predict(x_val)

In [115]:
print(f1_score(y_val, yhat_val))
confusion_matrix(y_val, yhat_val)

0.4194615021256495


array([[5224,  401],
       [ 828,  444]])

In [120]:
yhat_val_svm = svc.predict(x_val)
print(f1_score(y_val, yhat_val_svm))
confusion_matrix(y_val, yhat_val_svm)

0.12781408859840232


array([[5608,   17],
       [1184,   88]])

In [149]:
yhat_val_lgbm = lgbm_.predict(x_val)
print(f1_score(y_val, yhat_val_lgbm))
confusion_matrix(y_val, yhat_val_lgbm)

0.4438254410399257


array([[5221,  404],
       [ 794,  478]])

- us_yn

In [3]:
data.loc[data[data['us_yn']=='T'].index, 'us_yn'] = 'N' # 임시, 철거예정

In [7]:
no_energy_idx = data[(data['ele_engry_us_201401']==0) & (data['ele_engry_us_201404']==0) & (data['ele_engry_us_201407']==0) & (data['ele_engry_us_201410']==0)& (data['ele_engry_us_201412']==0)
    & (data['ele_engry_us_201501']==0)& (data['ele_engry_us_201504']==0) & (data['ele_engry_us_201507']==0) & (data['ele_engry_us_201510']==0)& (data['ele_engry_us_201512']==0)
    & (data['ele_engry_us_201601']==0)& (data['ele_engry_us_201604']==0) & (data['ele_engry_us_201607']==0) & (data['ele_engry_us_201610']==0)& (data['ele_engry_us_201612']==0)
    & (data['ele_engry_us_201701']==0)& (data['ele_engry_us_201704']==0) & (data['ele_engry_us_201707']==0) & (data['ele_engry_us_201710']==0)& (data['ele_engry_us_201712']==0)
    & (data['ele_engry_us_201801']==0)& (data['ele_engry_us_201804']==0) & (data['ele_engry_us_201807']==0) & (data['ele_engry_us_201810']==0)& (data['ele_engry_us_201812']==0)
    &(data['gas_engry_us_201401']==0) & (data['gas_engry_us_201404']==0) & (data['gas_engry_us_201407']==0) & (data['gas_engry_us_201410']==0)& (data['gas_engry_us_201412']==0)
    & (data['gas_engry_us_201501']==0)& (data['gas_engry_us_201504']==0) & (data['gas_engry_us_201507']==0) & (data['gas_engry_us_201510']==0)& (data['gas_engry_us_201512']==0)
    & (data['gas_engry_us_201601']==0)& (data['gas_engry_us_201604']==0) & (data['gas_engry_us_201607']==0) & (data['gas_engry_us_201610']==0)& (data['gas_engry_us_201612']==0)
    & (data['gas_engry_us_201701']==0)& (data['gas_engry_us_201704']==0) & (data['gas_engry_us_201707']==0) & (data['gas_engry_us_201710']==0)& (data['gas_engry_us_201712']==0)
    & (data['gas_engry_us_201801']==0)& (data['gas_engry_us_201804']==0) & (data['gas_engry_us_201807']==0) & (data['gas_engry_us_201810']==0)& (data['gas_engry_us_201812']==0)].index

In [8]:
data[(data['ele_engry_us_201401']==0) & (data['ele_engry_us_201404']==0) & (data['ele_engry_us_201407']==0) & (data['ele_engry_us_201410']==0)& (data['ele_engry_us_201412']==0)
    & (data['ele_engry_us_201501']==0)& (data['ele_engry_us_201504']==0) & (data['ele_engry_us_201507']==0) & (data['ele_engry_us_201510']==0)& (data['ele_engry_us_201512']==0)
    & (data['ele_engry_us_201601']==0)& (data['ele_engry_us_201604']==0) & (data['ele_engry_us_201607']==0) & (data['ele_engry_us_201610']==0)& (data['ele_engry_us_201612']==0)
    & (data['ele_engry_us_201701']==0)& (data['ele_engry_us_201704']==0) & (data['ele_engry_us_201707']==0) & (data['ele_engry_us_201710']==0)& (data['ele_engry_us_201712']==0)
    & (data['ele_engry_us_201801']==0)& (data['ele_engry_us_201804']==0) & (data['ele_engry_us_201807']==0) & (data['ele_engry_us_201810']==0)& (data['ele_engry_us_201812']==0)
    &(data['gas_engry_us_201401']==0) & (data['gas_engry_us_201404']==0) & (data['gas_engry_us_201407']==0) & (data['gas_engry_us_201410']==0)& (data['gas_engry_us_201412']==0)
    & (data['gas_engry_us_201501']==0)& (data['gas_engry_us_201504']==0) & (data['gas_engry_us_201507']==0) & (data['gas_engry_us_201510']==0)& (data['gas_engry_us_201512']==0)
    & (data['gas_engry_us_201601']==0)& (data['gas_engry_us_201604']==0) & (data['gas_engry_us_201607']==0) & (data['gas_engry_us_201610']==0)& (data['gas_engry_us_201612']==0)
    & (data['gas_engry_us_201701']==0)& (data['gas_engry_us_201704']==0) & (data['gas_engry_us_201707']==0) & (data['gas_engry_us_201710']==0)& (data['gas_engry_us_201712']==0)
    & (data['gas_engry_us_201801']==0)& (data['gas_engry_us_201804']==0) & (data['gas_engry_us_201807']==0) & (data['gas_engry_us_201810']==0)& (data['gas_engry_us_201812']==0)]

,dt_of_fr,fr_yn,bldng_us,bldng_archtctr,bldng_cnt,bldng_ar,ttl_ar,lnd_ar,dt_of_athrztn,ttl_grnd_flr,ttl_dwn_flr,bldng_us_clssfctn,tmprtr,prcpttn,wnd_spd,wnd_drctn,hmdt,gas_engry_us_201401,ele_engry_us_201401,gas_engry_us_201402,ele_engry_us_201402,gas_engry_us_201403,ele_engry_us_201403,gas_engry_us_201404,ele_engry_us_201404,gas_engry_us_201405,ele_engry_us_201405,gas_engry_us_201406,ele_engry_us_201406,gas_engry_us_201407,ele_engry_us_201407,gas_engry_us_201408,ele_engry_us_201408,gas_engry_us_201409,ele_engry_us_201409,gas_engry_us_201410,ele_engry_us_201410,gas_engry_us_201411,ele_engry_us_201411,gas_engry_us_201412,ele_engry_us_201412,gas_engry_us_201501,ele_engry_us_201501,gas_engry_us_201502,ele_engry_us_201502,gas_engry_us_201503,ele_engry_us_201503,gas_engry_us_201504,ele_engry_us_201504,gas_engry_us_201505,...,ele_engry_us_201809,gas_engry_us_201810,ele_engry_us_201810,gas_engry_us_201811,ele_engry_us_201811,gas_engry_us_201812,ele_engry_us_201812,lw_13101010,lw_13101110,lw_13101210,lw_13101211,lw_13101310,lw_13101410,lw_13111010,lw_13111110,lw_13121010,lw_13121011,lw_13131010,lw_13131110,lw_13141010,lw_13141011,jmk,id,rgnl_ar_nm,rgnl_ar_nm2,lnd_us_sttn_nm,rd_sd_nm,emd_nm,hm_cnt,fr_sttn_dstnc,bldng_ar_prc,fr_wthr_fclt_dstnc,fr_mn_cnt,mlt_us_yn,cctv_dstnc,fr_wthr_fclt_in_100m,cctv_in_100m,tbc_rtl_str_dstnc,sft_emrgnc_bll_dstnc,ahsm_dstnc,no_tbc_zn_dstnc,bldng_cnt_in_50m,trgt_crtr,fr_fghtng_fclt_spcl_css_5_yn,fr_fghtng_fclt_spcl_css_6_yn,us_yn,dngrs_thng_yn,slf_fr_brgd_yn,blk_dngrs_thng_mnfctr_yn,cltrl_hrtg_yn
0,2017-10-20 05:54,Y,단독주택,블록구조,3,69.42,69.42,0.00,1977,1.0,0.0,주거용,10.7,NaN,0.9,200.0,96.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,대,106536,자연녹지지역,지정되지않음,단독,세로한면(불),경상남도 창녕군 창녕읍,17360.0,4547,NaN,133,137.0,N,112,0,0,1891,29231,11322,88,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-10-30 14:57,Y,공동주택,철근콘크리트구조,1,583.80,2516.76,1446.00,2.00011e+07,5.0,0.0,주거용,15.5,NaN,2.0,90.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,답,107560,제2종일반주거지역,지정되지않음,연립,중로한면,경상남도 창녕군 창녕읍,17285.0,3340,618105.0,143,137.0,N,165,0,0,277,31197,12451,72,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-06-14 05:23,N,단독주택,일반목구조,2,48.92,48.92,0.00,1936,1.0,0.0,주거용,20.6,NaN,0.4,0.0,91.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,대,97550,제1종일반주거지역,지정되지않음,단독,세로한면(불),경상남도 창원시의창구 대산면,7327.0,179,719542.0,1585,176.0,N,131,0,0,438,4270,407,508,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2018-04-21 15:41,Y,단독주택,철근콘크리트구조,1,96.66,149.04,486.00,2.01509e+07,2.0,0.0,주거용,27.8,NaN,4.5,180.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,대,132645,자연녹지지역,지정되지않음,단독,세로한면(가),경상남도 창녕군 창녕읍,17278.0,4278,899087.0,294,137.0,N,210,0,0,1425,29709,11690,206,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2018-03-03 23:11,N,단독주택,벽돌구조,6,92.00,92.04,0.00,1.99411e+07,1.0,0.0,주거용,5.8,NaN,0.5,200.0,78.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,답,83207,계획관리지역,지정되지않음,단독,소로한면,경상남도 창녕군 창녕읍,17302.0,2008,233594.0,310,137.0,N,519,0,0,1951,33829,13237,695,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2014-12-08 05:23,N,단독주택,일반목구조,4,66.13,66.13,0.00,1947,1.0,0.0,주거용,-2.8,NaN,0.8,70.0,89.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [14]:
data.loc[no_energy_idx][['ttl_ar','bldng_ar','lnd_ar','jmk','bldng_us','bldng_archtctr','lnd_us_sttn_nm','rgnl_ar_nm','bldng_us_clssfctn']]

,ttl_ar,bldng_ar,lnd_ar,jmk,bldng_us,bldng_archtctr,lnd_us_sttn_nm,rgnl_ar_nm,bldng_us_clssfctn
0,69.42,69.42,0.00,대,단독주택,블록구조,단독,자연녹지지역,주거용
2,2516.76,583.80,1446.00,답,공동주택,철근콘크리트구조,연립,제2종일반주거지역,주거용
3,48.92,48.92,0.00,대,단독주택,일반목구조,단독,제1종일반주거지역,주거용
5,149.04,96.66,486.00,대,단독주택,철근콘크리트구조,단독,자연녹지지역,주거용
7,92.04,92.00,0.00,답,단독주택,벽돌구조,단독,계획관리지역,주거용
9,66.13,66.13,0.00,대,단독주택,일반목구조,단독,계획관리지역,주거용
10,98.96,98.96,310.00,대,단독주택,일반목구조,단독,제2종일반주거지역,주거용
13,129.55,44.22,82.00,대,단독주택,철근콘크리트구조,상업용,제2종일반주거지역,주거용
14,66.12,66.12,0.00,대,단독주택,일반목구조,단독,제2종일반주거지역,주거용
16,81.36,81.36,640.00,대,단독주택,블록구조,주거나지,제2종일반주거지역,주거용


In [12]:
data[(data['gas_engry_us_201401'].isnull())][['jmk','bldng_us','bldng_archtctr','lnd_us_sttn_nm','rgnl_ar_nm','bldng_us_clssfctn']]

,jmk,bldng_us,bldng_archtctr,lnd_us_sttn_nm,rgnl_ar_nm,bldng_us_clssfctn
1,대,NaN,NaN,단독,계획관리지역,NaN
4,임,NaN,NaN,자연림,자연환경보전지역,NaN
6,대,NaN,NaN,주거기타,계획관리지역,NaN
8,대,NaN,NaN,단독,자연녹지지역,NaN
11,대,NaN,NaN,단독,제2종일반주거지역,NaN
15,종,NaN,NaN,주거기타,보전관리지역,NaN
21,대,NaN,NaN,단독,자연녹지지역,NaN
22,대,NaN,NaN,단독,계획관리지역,NaN
25,대,NaN,NaN,단독,계획관리지역,NaN
27,대,NaN,NaN,단독,제2종일반주거지역,NaN


In [11]:
data['fr_fghtng_fclt_spcl_css_5_yn'].value_counts()

N    8437
Name: fr_fghtng_fclt_spcl_css_5_yn, dtype: int64

In [105]:
# data[data['trgt_crtr'].isna()] # 결측치는 소방점검 대상기준이 아님

In [20]:
data['trgt_crtr'].value_counts()

일반대상물                        7589
민박7실이하                        294
자동화재탐지설치대상                    146
옥내소화전설치대상                      69
스프링클러,물분무등설치대상                 28
민박7실이상                         16
펜션7실이하                         12
공동주택(주택법시행령제63조제1항각호1에해당)       6
연면적 15,000 이상                   6
11층이상                           2
펜션7실이상                          2
기타(가설건축물분류)                     2
주거용비닐하우스                        1
콘테이너하우스                         1
임시사무실,창고,숙소                     1
기타                              1
견본주택(모델하우스)                     1
Name: trgt_crtr, dtype: int64

- lw___

In [5]:
data[data['ttl_ar']>4000][['lw_13101010','lw_13101110', 'lw_13101210','lw_13101211','lw_13101310','lw_13101410','lw_13111010','lw_13111110','lw_13121010','lw_13121011','lw_13131010','lw_13131110','lw_13141010','lw_13141011']]

,lw_13101010,lw_13101110,lw_13101210,lw_13101211,lw_13101310,lw_13101410,lw_13111010,lw_13111110,lw_13121010,lw_13121011,lw_13131010,lw_13131110,lw_13141010,lw_13141011
12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
206,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
241,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
279,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
351,4.00,3.48,4.000000,4.000000,4.000000,2.000000,4.000000,0.000000,0.0,0.0,3.48,3.48,0.00,0.00
357,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
369,3.65,4.00,4.000000,0.000000,4.000000,1.615385,4.000000,4.000000,0.0,0.0,3.00,3.00,4.65,4.65


In [13]:
data.isnull().sum().sort_values(ascending=False)

bldng_ar                9660
ttl_ar                  8426
lnd_us_sttn_nm          1776
rd_sd_nm                1491
rgnl_ar_nm              1491
rgnl_ar_nm2             1491
hm_cnt                   701
wnd_drctn                221
fr_mn_cnt                 41
wnd_spd                   33
hmdt                      22
tmprtr                    13
emd_nm                     4
bldng_cnt                  0
jmk                        0
id                         0
fr_yn                      0
bldng_cnt_in_50m           0
no_tbc_zn_dstnc            0
fr_sttn_dstnc              0
fr_wthr_fclt_dstnc         0
mlt_us_yn                  0
cctv_dstnc                 0
fr_wthr_fclt_in_100m       0
cctv_in_100m               0
tbc_rtl_str_dstnc          0
sft_emrgnc_bll_dstnc       0
ahsm_dstnc                 0
dt_of_fr                   0
dtype: int64

-  ttl_dwn_flr , ttl_grnd_flr

In [13]:
data[data['ttl_grnd_flr'].isnull()]['lnd_us_sttn_nm'].value_counts()

단독         3024
주거나지        295
도로등         290
주거기타        236
과수원         162
전기타         137
답기타         115
하천등         115
토지임야         75
공업용          70
상업용          69
전            69
주상용          54
조림           47
임야기타         27
주상기타         23
답            21
공원등          17
공업기타         15
상업나지         15
업무용          14
공업나지         13
주차장등         11
아파트          10
기타            9
주상나지          8
운동장등          6
유해.혐오시설       6
상업기타          6
다세대           5
연립            3
목장용지          2
유원지           1
특수기타          1
위험시설          1
발전소           1
골프장 회원제       1
Name: lnd_us_sttn_nm, dtype: int64

In [4]:
data.loc[data[(data['jmk'].isin(['답','전'])) & (data['ttl_dwn_flr'].isnull()) & (data['lnd_us_sttn_nm'].isin(['전','답']))].index, 'ttl_dwn_flr'] = 0
data.loc[data[(data['jmk'].isin(['답','전'])) & (data['ttl_grnd_flr'].isnull())& (data['lnd_us_sttn_nm'].isin(['전','답']))].index, 'ttl_grnd_flr'] = 0
data.loc[data[(data['bldng_ar']==0) & (data['ttl_grnd_flr'].isnull())& (data['lnd_us_sttn_nm']=='자연림')].index, 'ttl_grnd_flr'] = 0
data.loc[data[(data['bldng_ar']==0) & (data['ttl_dwn_flr'].isnull())& (data['lnd_us_sttn_nm']=='자연림')].index, 'ttl_dwn_flr'] = 0

In [5]:
data.loc[data[(data['ttl_grnd_flr'].isnull()) & (data['bldng_ar'] == data['ttl_ar']) & (data['bldng_ar']!=0)].index, 'ttl_grnd_flr'] = 0
data.loc[data[(data['ttl_dwn_flr'].isnull()) & (data['bldng_ar'] == data['ttl_ar']) & (data['bldng_ar']!=0)].index, 'ttl_dwn_flr'] = 0

In [6]:
data.loc[data[(data['ttl_dwn_flr'].isnull()) & (data['bldng_ar'] > data['ttl_ar'])].index,'ttl_dwn_flr'] = 0 
data.loc[data[(data['ttl_grnd_flr'].isnull()) & (data['bldng_ar']*2 > data['ttl_ar'])].index,'ttl_grnd_flr'] = 2 
data.loc[data[(data['ttl_grnd_flr'].isnull()) & (data['bldng_ar']*3 > data['ttl_ar'])].index,'ttl_grnd_flr'] = 3

In [7]:
data[(data['ttl_dwn_flr'].isnull()) & (data['bldng_ar'] > data['ttl_ar'])][['bldng_ar','lnd_ar','ttl_ar','bldng_cnt','jmk','lnd_us_sttn_nm','bldng_us','bldng_archtctr','rgnl_ar_nm','bldng_us_clssfctn']]

,bldng_ar,lnd_ar,ttl_ar,bldng_cnt,jmk,lnd_us_sttn_nm,bldng_us,bldng_archtctr,rgnl_ar_nm,bldng_us_clssfctn


- lnd_us_sttn_nm

In [14]:
data.dropna(inplace=True)

In [15]:
data.drop(columns=['id','rgnl_ar_nm2'], inplace=True)